In [3]:
from pyalex import (
    Works, Authors, Sources,
    Institutions, Concepts, Publishers, Funders
)
import pyalex
import pandas as pd
import numpy as np
pyalex.config.email = "david@rs21.io"

from flair.embeddings import DocumentPoolEmbeddings
from flair.data import Sentence
from flair.embeddings import SentenceTransformerDocumentEmbeddings

EMBEDDING_MODEL_1 = "all-mpnet-base-v2" 

# this one is also good: all-MiniLM-L6-v2
EMBEDDING_MODEL_2 = "all-MiniLM-L6-v2"
SENT_EMBEDDINGS_1 = SentenceTransformerDocumentEmbeddings(EMBEDDING_MODEL_1)
SENT_EMBEDDINGS_2 = SentenceTransformerDocumentEmbeddings(EMBEDDING_MODEL_2)
DOC_EMBEDDINGS= DocumentPoolEmbeddings([SENT_EMBEDDINGS_2])

import torch
from tqdm import tqdm
import yake
import umap.umap_ as umap
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture as GMM
import altair as alt
import math
import plotly.express as px
import textwrap
import streamlit as st

In [4]:
import os
os.environ["OPENAI_API_KEY"] = st.secrets["OPENAI_API_KEY"]

2024-05-30 14:30:58.895 INFO    streamlit.runtime.secrets: Secrets found in multiple locations: /home/davidd/.streamlit/secrets.toml, /home/davidd/2023/techintel-quantum/.streamlit/secrets.toml. When multiple secret.toml files exist, local secrets will take precedence over global secrets.


In [5]:
search_term = 'missile'
robot_concepts = Concepts().search_filter(display_name=search_term).get()
len(robot_concepts)
for i in range(len(robot_concepts)):
    id_, display_name = robot_concepts[i]['id'], robot_concepts[i]['display_name']
    print(id_, display_name)

https://openalex.org/C2778857364 Missile
https://openalex.org/C522053795 Missile guidance
https://openalex.org/C122136912 Ballistic missile
https://openalex.org/C559253537 Missile defense
https://openalex.org/C28849524 Cruise missile
https://openalex.org/C202802212 Air-to-air missile


In [6]:
results, meta  = Concepts().get(return_meta=True)
print(meta)

{'count': 65073, 'db_response_time_ms': 31, 'page': 1, 'per_page': 25, 'groups_count': None}


In [7]:
search_term = 'jamming'
search_term = 'radar jamming and deception|electronic warfare|Network-centric warfare|Air-to-air missile'
search_term = ('radar jamming and deception|electronic warfare|Network-centric warfare|missile guidance' +
               '|Robot manipulator|Automatic target recognition' + 
              '|Pulsed power|High-energy X-rays' + 
              '|Ballistic missile|Air-to-air missile' + 
              '|terminal guidance')
jamming_concepts = Concepts().\
search_filter(display_name=search_term).get()

In [8]:
search_term = 'quantum computation|quantum computers|quantum sensors|quantum cryptography'
quantum_concepts = Concepts().\
search_filter(display_name=search_term).get()

In [9]:
concepts = []
for i in range(len(quantum_concepts)):
    id_, display_name = quantum_concepts[i]['id'], quantum_concepts[i]['display_name']
    concepts.append((id_, display_name))
concepts

[('https://openalex.org/C58053490', 'Quantum computer'),
 ('https://openalex.org/C144901912', 'Quantum cryptography'),
 ('https://openalex.org/C89143813', 'Quantum sensor'),
 ('https://openalex.org/C187483380', 'Topological quantum computer'),
 ('https://openalex.org/C192353077', 'Adiabatic quantum computation'),
 ('https://openalex.org/C183968085', 'Trapped ion quantum computer'),
 ('https://openalex.org/C108277079', 'Post-quantum cryptography'),
 ('https://openalex.org/C19271316', 'One-way quantum computer')]

In [10]:
def process_works_list(worklist:list):
    """
    transforms the 
    works list into a dataframe.
    """
    abstracts_dict = {h["id"]:h["abstract"] for h in worklist}
    df = pd.DataFrame.from_records(worklist)
    try: 
        del df['abstract_inverted_index'] # though don't all have abstracts is the problem
        df['abstract'] = df['id'].map(abstracts_dict)
    except:
        pass
   # df['author_affils'] = df['authorships'].apply(get_authors_and_affils)
    return df

In [11]:
for i in range(len(quantum_concepts)):
    print(quantum_concepts[i]['id'], quantum_concepts[i]['works_count'])

https://openalex.org/C58053490 62241
https://openalex.org/C144901912 11270
https://openalex.org/C89143813 8177
https://openalex.org/C187483380 3369
https://openalex.org/C192353077 1964
https://openalex.org/C183968085 1212
https://openalex.org/C108277079 1384
https://openalex.org/C19271316 1536


In [12]:
def get_hpm_frame():
    #hpm_pager = Works().filter(publication_year='>2020').search("high power microwave").\
#filter(authorships={"institutions":{"country_code":"CN"}}).paginate(per_page=200,
#                                                                    n_max=None)
    hpm_pager = Works().filter(publication_year='>2020').search("high power microwave").\
        paginate(per_page=200,  n_max=None)
    df = pd.DataFrame()
    for page in tqdm(hpm_pager):
        dfpage = process_works_list(page)
        df = pd.concat([df, dfpage], ignore_index=True)
        df.drop_duplicates(subset='id', keep='first',inplace=True)
    return df  

In [13]:
def get_sbl_frame():
    #sbl_pager = Works().filter(publication_year='>2020').search("space based laser").\
#filter(authorships={"institutions":{"country_code":"CN"}}).paginate(per_page=200,
#                                                                    n_max=None)
    sbl_pager = Works().filter(publication_year='>2020').search("space based laser").\
        paginate(per_page=200,
                                                                    n_max=None)
    df = pd.DataFrame()
    for page in tqdm(sbl_pager):
        dfpage = process_works_list(page)
        df = pd.concat([df, dfpage], ignore_index=True)
        df.drop_duplicates(subset='id', keep='first',inplace=True)
    return df    

In [14]:
def get_kkv_frame():
    #kkv_pager = Works().filter(publication_year='>2020').search("kinetic kill vehicle").\
#filter(authorships={"institutions":{"country_code":"CN"}}).paginate(per_page=200,
#                                                                    n_max=None)
    kkv_pager = Works().filter(publication_year='>2020').search("kinetic kill vehicle").\
        paginate(per_page=200,
                                                                    n_max=None)
    df = pd.DataFrame()
    for page in tqdm(kkv_pager):
        dfpage = process_works_list(page)
        df = pd.concat([df, dfpage], ignore_index=True)
        df.drop_duplicates(subset='id', keep='first',inplace=True)
    return df                                                               
    

In [15]:
def get_rka_frame():
   # rka_pager = Works().filter(publication_year='>2020').search("relativistic klystron amplifier").\
#filter(authorships={"institutions":{"country_code":"CN"}}).paginate(per_page=200,
#                                                                    n_max=None)
    rka_pager = Works().filter(publication_year='>2020').search("relativistic klystron amplifier").\
        paginate(per_page=200,
                                                                    n_max=None)
    df = pd.DataFrame()
    for page in tqdm(rka_pager):
        dfpage = process_works_list(page)
        df = pd.concat([df, dfpage], ignore_index=True)
        df.drop_duplicates(subset='id', keep='first',inplace=True)
    return df 

In [16]:
def get_concept_frame(concepts_list:list, i:int):
    """
    takes a list of Concepts() results and an index
    and forms the pagination object to retrive the 
    records
    """
    pager = Works().filter(publication_year='>2020',
    #concepts={"id":f"{concepts_list[i]['id']}"}).filter(authorships={"institutions":{"country_code":"CN"}}).\
    #paginate(per_page=200,n_max=None)
    concepts={"id":f"{concepts_list[i]['id']}"}).\
    paginate(per_page=200,n_max=None)
    df = pd.DataFrame()
    for page in tqdm(pager):
        dfpage = process_works_list(page)
        df = pd.concat([df, dfpage], ignore_index=True)
        df.drop_duplicates(subset='id', keep='first',inplace=True)
    return df

In [18]:
frames_list = []
for i in range(len(quantum_concepts)):
    df = get_concept_frame(quantum_concepts, i)
    frames_list.append(df)

114it [03:32,  1.86s/it]
16it [00:24,  1.54s/it]
16it [00:25,  1.60s/it]
5it [00:07,  1.52s/it]
3it [00:03,  1.10s/it]
3it [00:03,  1.05s/it]
4it [00:05,  1.32s/it]
3it [00:03,  1.13s/it]


In [19]:
frames_list[3]['created_date'].max()

'2024-05-22'

In [20]:
frames_list[2]['abstract'].value_counts(dropna=False)

None                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [21]:
len(frames_list)

8

In [22]:
dftop = pd.concat(frames_list,
                  ignore_index=True)
dftop.drop_duplicates(subset='id', keep='first', 
                      inplace=True)

dftop.set_index('id', inplace=True, drop=False)

dfall = dftop
print(dfall.shape)

dfall['content'] = dfall['title'] + ". " + dfall['abstract']

dfrecords = dfall[~dfall['content'].isna()].copy()

(27174, 49)


In [23]:
def get_keywords(text:str, top:int=7, stopwords=None):
    """
    takes a blob of text and 
    returns the top **top** 
    keywords as a list
    """
    kw_extractor = yake.KeywordExtractor(top=top, stopwords=stopwords)
    keywords = kw_extractor.extract_keywords(text)
    return [p[0] for p in keywords]

In [24]:
def get_top_concepts(concept_list:list,score:float=.6):
    """
    takes a list of concept dictionaries 
    returns the top **top** display_names;
    concepts whose score is >= score
    """
    return [c['display_name'] for c in concept_list if c['score'] >= score]

In [25]:
dfrecords['keywords'] = dfrecords['content'].apply(get_keywords)
dfrecords['top_concepts'] = dfrecords['concepts'].apply(get_top_concepts)

In [27]:
jamming_concepts = quantum_concepts

In [28]:
texts = dfrecords['content'].str.lower().values.tolist()

In [29]:
def get_content_embeddings(dfrecords:pd.DataFrame) -> pd.DataFrame:
    """
    passes the preprocessed mitigation strings
    data through the embedding model to produce the vector
    space representation of each pet mitigation.
    """
    sent = Sentence("The grass is green.")
    DOC_EMBEDDINGS.embed(sent)
    texts = dfrecords["content"].str.lower().values.tolist()
    all_descriptions = np.empty((len(texts), len(sent.embedding)))
    for i in tqdm(range(len(texts))):
        sent = Sentence(texts[i])
        DOC_EMBEDDINGS.embed(sent)
        all_descriptions[i, :] = sent.embedding.cpu().numpy()
        # gc.collect()
        torch.cuda.empty_cache()
    dfcontentvectors = pd.DataFrame.from_records(all_descriptions, index=dfrecords.index)
    return dfcontentvectors

In [30]:
dfcontentvectors = get_content_embeddings(dfrecords)

100%|███████████████████████████████████████████████████████████████████| 24359/24359 [06:40<00:00, 60.76it/s]


In [31]:
#umap.UMAP?
N_COMPONENTS = 2 # can visualize this way
umap_reducer = umap.UMAP(n_components=N_COMPONENTS,
                       #  metric='euclidean')
                         random_state=1234,
                         metric='cosine')  # can experiment with this metric as well as the other 
# parameters
# to see what other literature is in the same information space, we need to keep this umap_reducer 
# object as well as the gmm model below.

# Apply UMAP to the vectorized strings
reduced_vectors = umap_reducer.fit_transform(dfcontentvectors.to_numpy())
dfreduced = pd.DataFrame.from_records(reduced_vectors, 
                index=dfcontentvectors.index)
dfreduced.columns = ['x','y']

/home/davidd/.local/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


# use hdbscan to cluster

In [32]:
import hdbscan

hdbscan_args = {'min_cluster_size': 15,
                            'metric': 'euclidean',
                            'cluster_selection_method': 'eom',
                            'cluster_selection_epsilon': 0.1
               }

cluster = hdbscan.HDBSCAN(**hdbscan_args).fit(dfreduced[['x','y']].to_numpy())

dfreduced['cluster'] = cluster.labels_
dfreduced['probability'] = cluster.probabilities_

dfpapers = dfrecords.merge(dfreduced, left_index=True,
                           right_index=True)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [33]:
#help(dfpapers.explode)
del dfpapers['id']
dfstart = dfpapers.reset_index()
dfstart.head()

,id,doi,title,display_name,publication_year,publication_date,ids,language,primary_location,type,...,updated_date,created_date,is_authors_truncated,abstract,content,top_concepts,x,y,cluster,probability
0,https://openalex.org/W3189250281,https://doi.org/10.1038/s42254-021-00348-9,Variational quantum algorithms,Variational quantum algorithms,2021,2021-08-12,{'openalex': 'https://openalex.org/W3189250281...,en,"{'is_oa': False, 'landing_page_url': 'https://...",review,...,2024-05-27T05:35:26.432933,2021-08-16,NaN,Applications such as simulating complicated qu...,Variational quantum algorithms. Applications s...,"[Quantum computer, Computer science, Quantum a...",6.192506,-3.169462,145,1.000000
1,https://openalex.org/W1521266393,https://doi.org/10.1103/revmodphys.93.025005,Circuit quantum electrodynamics,Circuit quantum electrodynamics,2021,2021-05-19,{'openalex': 'https://openalex.org/W1521266393...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,2024-05-22T13:15:01.185356,2016-06-24,NaN,This review surveys the development over the l...,Circuit quantum electrodynamics. This review s...,"[Physics, Circuit quantum electrodynamics, Cav...",6.635708,2.777389,70,0.665913
2,https://openalex.org/W4213212652,https://doi.org/10.1103/revmodphys.94.015004,Noisy intermediate-scale quantum algorithms,Noisy intermediate-scale quantum algorithms,2022,2022-02-15,{'openalex': 'https://openalex.org/W4213212652...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,2024-05-22T08:21:09.828213,2022-02-24,NaN,A universal fault-tolerant quantum computer th...,Noisy intermediate-scale quantum algorithms. A...,"[Computer science, Quantum computer, Qubit]",5.223140,-0.724157,143,0.980064
3,https://openalex.org/W3173899054,https://doi.org/10.1103/physrevlett.127.180501,Strong Quantum Computational Advantage Using a...,Strong Quantum Computational Advantage Using a...,2021,2021-10-25,{'openalex': 'https://openalex.org/W3173899054...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,2024-05-29T18:15:50.250347,2021-07-05,NaN,Scaling up to a large number of qubits with hi...,Strong Quantum Computational Advantage Using a...,"[Quantum computer, Qubit, Computer science]",5.762541,-0.747414,-1,0.000000
4,https://openalex.org/W3180039530,https://doi.org/10.1038/s41586-021-03582-4,Quantum phases of matter on a 256-atom program...,Quantum phases of matter on a 256-atom program...,2021,2021-07-07,{'openalex': 'https://openalex.org/W3180039530...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,2024-05-27T20:40:46.275097,2021-07-19,NaN,Motivated by far-reaching applications ranging...,Quantum phases of matter on a 256-atom program...,"[Quantum simulator, Quantum]",8.667332,2.101456,78,0.847872


In [34]:
dfstart['publication_year'].value_counts(dropna=False)

2023    8682
2022    7259
2021    6654
2024    1764
Name: publication_year, dtype: int64

In [35]:
dfstart.shape

(24359, 55)

In [36]:
dfbig = dfstart.explode(column='authorships')
dfbig.shape, dfstart.shape

((112681, 55), (24359, 55))

In [37]:
dfbig.columns

Index(['id', 'doi', 'title', 'display_name', 'publication_year',
       'publication_date', 'ids', 'language', 'primary_location', 'type',
       'type_crossref', 'indexed_in', 'open_access', 'authorships',
       'countries_distinct_count', 'institutions_distinct_count',
       'corresponding_author_ids', 'corresponding_institution_ids', 'apc_list',
       'apc_paid', 'has_fulltext', 'fulltext_origin', 'cited_by_count',
       'cited_by_percentile_year', 'biblio', 'is_retracted', 'is_paratext',
       'primary_topic', 'topics', 'keywords', 'concepts', 'mesh',
       'locations_count', 'locations', 'best_oa_location',
       'sustainable_development_goals', 'grants', 'datasets', 'versions',
       'referenced_works_count', 'referenced_works', 'related_works',
       'ngrams_url', 'cited_by_api_url', 'counts_by_year', 'updated_date',
       'created_date', 'is_authors_truncated', 'abstract', 'content',
       'top_concepts', 'x', 'y', 'cluster', 'probability'],
      dtype='object')

In [38]:
dfbig.locations.iloc[68]

[{'is_oa': False,
  'landing_page_url': 'https://doi.org/10.1103/physrevlett.127.180501',
  'pdf_url': None,
  'source': {'id': 'https://openalex.org/S24807848',
   'display_name': 'Physical review letters',
   'issn_l': '0031-9007',
   'issn': ['0031-9007', '1079-7114', '1092-0145'],
   'is_oa': False,
   'is_in_doaj': False,
   'host_organization': 'https://openalex.org/P4310320261',
   'host_organization_name': 'American Physical Society',
   'host_organization_lineage': ['https://openalex.org/P4310320261'],
   'host_organization_lineage_names': ['American Physical Society'],
   'type': 'journal'},
  'license': None,
  'license_id': None,
  'version': None,
  'is_accepted': False,
  'is_published': False},
 {'is_oa': True,
  'landing_page_url': 'https://arxiv.org/abs/2106.14734',
  'pdf_url': 'https://arxiv.org/pdf/2106.14734',
  'source': {'id': 'https://openalex.org/S4306400194',
   'display_name': 'arXiv (Cornell University)',
   'issn_l': None,
   'issn': None,
   'is_oa': True,

In [39]:
def add_extra_to_authorships(row: pd.DataFrame):
    """
    row[authorships] is a dictionary;
    add in the id key to that dictionary
    whose value is row[id]
    """
    complete_dict = row["authorships"]
   # assert type(complete_dict) == dict
    #print(type(complete_dict))
    if type(complete_dict) == dict:
        complete_dict["id"] = row["id"]
        complete_dict["x"] = row["x"]
        complete_dict["y"] = row["y"]
        complete_dict["cluster"] = row["cluster"]
        complete_dict["cluster_score"] = row["probability"]
        complete_dict["title"] = row["title"]
        complete_dict["abstract"] = row["abstract"]
        complete_dict["doi"] = row["doi"]
        complete_dict["publication_date"] = row["publication_date"]
        complete_dict["publication_year"] = row["publication_year"]
        complete_dict["grants"] = row["grants"]
        complete_dict["locations"] = row["locations"]
        return complete_dict
    else:
        return row["authorships"]

In [40]:
dfbig['big_authorships'] = dfbig.apply(add_extra_to_authorships, axis=1)

In [41]:
dfbig.columns

Index(['id', 'doi', 'title', 'display_name', 'publication_year',
       'publication_date', 'ids', 'language', 'primary_location', 'type',
       'type_crossref', 'indexed_in', 'open_access', 'authorships',
       'countries_distinct_count', 'institutions_distinct_count',
       'corresponding_author_ids', 'corresponding_institution_ids', 'apc_list',
       'apc_paid', 'has_fulltext', 'fulltext_origin', 'cited_by_count',
       'cited_by_percentile_year', 'biblio', 'is_retracted', 'is_paratext',
       'primary_topic', 'topics', 'keywords', 'concepts', 'mesh',
       'locations_count', 'locations', 'best_oa_location',
       'sustainable_development_goals', 'grants', 'datasets', 'versions',
       'referenced_works_count', 'referenced_works', 'related_works',
       'ngrams_url', 'cited_by_api_url', 'counts_by_year', 'updated_date',
       'created_date', 'is_authors_truncated', 'abstract', 'content',
       'top_concepts', 'x', 'y', 'cluster', 'probability', 'big_authorships'],
      

In [42]:
dfbig['authorships'].iloc[69] # raw_affiliation_strings    

{'author_position': 'middle',
 'author': {'id': 'https://openalex.org/A5083638025',
  'display_name': 'Jiabing Yu',
  'orcid': 'https://orcid.org/0000-0001-7206-6592'},
 'institutions': [{'id': 'https://openalex.org/I4210103894',
   'display_name': 'Hefei National Center for Physical Sciences at Nanoscale',
   'ror': 'https://ror.org/01jeedh73',
   'country_code': 'CN',
   'type': 'facility',
   'lineage': ['https://openalex.org/I126520041',
    'https://openalex.org/I19820366',
    'https://openalex.org/I4210103894']},
  {'id': 'https://openalex.org/I126520041',
   'display_name': 'University of Science and Technology of China',
   'ror': 'https://ror.org/04c4dkn09',
   'country_code': 'CN',
   'type': 'education',
   'lineage': ['https://openalex.org/I126520041',
    'https://openalex.org/I19820366']},
  {'id': 'https://openalex.org/I4210150858',
   'display_name': 'Beijing Academy of Quantum Information Sciences',
   'ror': 'https://ror.org/04nqf9k60',
   'country_code': 'CN',
   't

In [43]:
#dfbig['authorships'].tolist()
bigvals = dfbig['authorships'].tolist()

In [44]:
dictvals = [c for c in bigvals if type(c) != float]

In [45]:
dictvals[0]['author'].keys()

dict_keys(['id', 'display_name', 'orcid'])

raw_affiliation_string -> raw_affiliation_strings

In [46]:
dftriple = pd.json_normalize(dictvals,
                  record_path=['institutions'],
                  meta=['id','raw_affiliation_strings','author_position', 'doi',
                        'title','abstract','publication_date', 'publication_year',
                        'grants','locations',
                        'is_corrresponding','x','y','cluster','cluster_score',
                       ['author','id'], ['author', 'display_name'],
                       ['author','orcid']],
                  errors='ignore',
                  sep='_',
                  meta_prefix='paper_',
                #  record_prefix='author_'
                 )

In [47]:
dftopics = dfcontentvectors.copy()
dftopics['cluster'] = dfpapers['cluster']
dfmeantopics = dftopics.groupby('cluster').mean().copy()
reduced_topics = umap_reducer.transform(dfmeantopics.to_numpy())
df_reduced_topics = pd.DataFrame.from_records(reduced_topics, 
                index=dfmeantopics.index)
df_reduced_topics.columns = ['x','y']
df_reduced_topics['topic'] = df_reduced_topics.index
df_reduced_topics.head()

def get_cluster_concepts(topic_num:int, n:int=20):
    """
    takes an integer topic_num corresponding to a 
    given topic number and
    returns the list of top n occuring concepts
    from the top_concept field
    """
    top_concepts = dfpapers[dfpapers['cluster'] == topic_num]['top_concepts'].tolist()
    flat_concepts = [item for sublist in top_concepts for item in sublist]
    concepts_dict = {c:flat_concepts.count(c) for c in flat_concepts}
    sorted_concepts = sorted(concepts_dict.items(), key=lambda x:x[1], reverse=True)
    return [c[0] for c in sorted_concepts][:n]

def get_yake_cluster_phrases(topic_num:int, n:int=20):
    """
    takes in an integer n corresponding
    to a given topic number and
    returns the list of keyphrases (TopicRank method)
    """
    documents = dfpapers[dfpapers['cluster'] == topic_num]['content'].tolist()
    topic_input = ". ".join(documents)
    #extractor = pke.unsupervised.TextRank()
    kw_extractor = yake.KeywordExtractor(top=n, stopwords=None)
    keywords = kw_extractor.extract_keywords(topic_input)
    #extractor.load_document(input=topic_input,
    #                    language='en',
    #                    normalization=None)

    #extractor.candidate_selection()

    #window = 2
    #use_stems = False
    #extractor.candidate_weighting(window=window,
    #                          use_stems=use_stems)
    #extractor.candidate_weighting()
    #threshold = 0.8
   # keyphrases = extractor.get_n_best(n=20, threshold=threshold)
    #keyphrases = extractor.get_n_best(n=n)
    return [p[0] for p in keywords]

wikiconcepts = df_reduced_topics['topic'].apply(get_cluster_concepts)

wikikeywords = df_reduced_topics['topic'].apply(get_yake_cluster_phrases)

dfpapers['id'] = dfpapers.index
dfinfo = dfpapers[['x','y','id','title','doi','cluster','grants',
                   'locations',
                 'publication_date','keywords','top_concepts']].copy()

centroids = dfinfo.groupby('cluster')[['x','y']].mean().copy()
centroids['concepts'] = wikiconcepts
centroids['cluster'] = centroids.index
centroids['keywords'] = wikikeywords

In [48]:
def wrap_it(x):
    return "<br>".join(textwrap.wrap(x, width=40))
   # return "<br>".join(textwrap.wrap(x.replace(r'\s+', ' '), width=40))

In [49]:
centroids['wrapped_keywords'] = centroids['keywords'].apply(str).apply(wrap_it)
centroids['wrapped_concepts'] = centroids['concepts'].apply(str).apply(wrap_it)

In [50]:
centroids.to_pickle('updatejammingcentroids2d.pkl')

In [51]:
dftriple.to_pickle('updatejammingdftriple2d.pkl')

In [52]:
def get_affils_cluster_sort(dc:pd.DataFrame, cl:int):
    """
    restricts the dataframe dc to cluster value cl
    and returns the results grouped by id, ror sorted
    by the some of probablity descending
    """
    # https://learning.oreilly.com/library/view/streamlit-for-data/9781803248226/text/ch004.xhtml
    dg = dc[dc['paper_cluster'] == cl].copy()
    print(cl)
    dv = dg.groupby(['id','display_name','country_code',
                     'type'])['paper_cluster_score'].sum().to_frame()
    dv.sort_values('paper_cluster_score', ascending=False, inplace=True)
    dv.reset_index(inplace=True) # map the display_name column with the geo_dict to get lattitude, longitude
    dv['latitude'] = dv['display_name'].apply(lambda x: affil_geo_dict.get(x, (None, None))[0])
    dv['longitude'] = dv['display_name'].apply(lambda x: affil_geo_dict.get(x, (None, None))[1])
    kw = centroids[centroids.cluster == cl]['keywords'].iloc[0]
    return dv, kw

In [56]:
#dv84, kw84 = get_affils_cluster_sort(dftriple, 1)
#print(kw84)
#dv84.head(10)

In [57]:
#dv84, kw84 = get_affils_cluster_sort(dftriple, 0)
#print(kw84)
#dv84.head(10)

In [58]:
#dv84, kw84 = get_affils_cluster_sort(dftriple, 0)
#print(kw84)
#dv84.head(20)

In [59]:
dfinfo = dfpapers[['x','y','id','title','doi','cluster','probability',
                 'publication_date','grants','locations',
                   'keywords','top_concepts']].copy()

In [60]:
dfpapers['primary_location'].iloc[58]

{'is_oa': True,
 'landing_page_url': 'https://doi.org/10.1038/s41928-022-00727-9',
 'pdf_url': 'https://www.nature.com/articles/s41928-022-00727-9.pdf',
 'source': {'id': 'https://openalex.org/S4210239724',
  'display_name': 'Nature electronics.',
  'issn_l': '2520-1131',
  'issn': ['2520-1131'],
  'is_oa': False,
  'is_in_doaj': False,
  'host_organization': 'https://openalex.org/P4310319908',
  'host_organization_name': 'Nature Portfolio',
  'host_organization_lineage': ['https://openalex.org/P4310319908',
   'https://openalex.org/P4310319965'],
  'host_organization_lineage_names': ['Nature Portfolio', 'Springer Nature'],
  'type': 'journal'},
 'license': 'cc-by',
 'license_id': 'https://openalex.org/licenses/cc-by',
 'version': 'publishedVersion',
 'is_accepted': True,
 'is_published': True}

In [61]:
dfpapers['locations'].iloc[58]

[{'is_oa': True,
  'landing_page_url': 'https://doi.org/10.1038/s41928-022-00727-9',
  'pdf_url': 'https://www.nature.com/articles/s41928-022-00727-9.pdf',
  'source': {'id': 'https://openalex.org/S4210239724',
   'display_name': 'Nature electronics.',
   'issn_l': '2520-1131',
   'issn': ['2520-1131'],
   'is_oa': False,
   'is_in_doaj': False,
   'host_organization': 'https://openalex.org/P4310319908',
   'host_organization_name': 'Nature Portfolio',
   'host_organization_lineage': ['https://openalex.org/P4310319908',
    'https://openalex.org/P4310319965'],
   'host_organization_lineage_names': ['Nature Portfolio', 'Springer Nature'],
   'type': 'journal'},
  'license': 'cc-by',
  'license_id': 'https://openalex.org/licenses/cc-by',
  'version': 'publishedVersion',
  'is_accepted': True,
  'is_published': True},
 {'is_oa': True,
  'landing_page_url': 'https://arxiv.org/abs/2101.12650',
  'pdf_url': 'https://arxiv.org/pdf/2101.12650',
  'source': {'id': 'https://openalex.org/S4306400

In [62]:
dftriple.columns

Index(['id', 'display_name', 'ror', 'country_code', 'type', 'lineage',
       'paper_id', 'paper_raw_affiliation_strings', 'paper_author_position',
       'paper_doi', 'paper_title', 'paper_abstract', 'paper_publication_date',
       'paper_publication_year', 'paper_grants', 'paper_locations',
       'paper_is_corrresponding', 'paper_x', 'paper_y', 'paper_cluster',
       'paper_cluster_score', 'paper_author_id', 'paper_author_display_name',
       'paper_author_orcid'],
      dtype='object')

In [63]:
dftriple[['paper_id','paper_raw_affiliation_strings']].head()

,paper_id,paper_raw_affiliation_strings
0,https://openalex.org/W3189250281,"[Theoretical Division, Los Alamos National Lab..."
1,https://openalex.org/W3189250281,"[Theoretical Division, Los Alamos National Lab..."
2,https://openalex.org/W3189250281,"[Google Quantum AI Team, Venice, CA, USA]"
3,https://openalex.org/W3189250281,"[Department of Materials, University of Oxford..."
4,https://openalex.org/W3189250281,"[NTT Secure Platform Laboratories, NTT Corpora..."


In [64]:
dftriple['paper_raw_affiliation_strings'].value_counts(dropna=False)

[Google Research, Mountain View, CA, USA]                                                                                                                                                                                                                                                                                                                                                                        473
[Hefei National Laboratory for Physical Sciences at the Microscale and Department of Modern Physics, University of Science and Technology of China, Hefei 230026, China, Shanghai Branch, CAS Center for Excellence in Quantum Information and Quantum Physics, University of Science and Technology of China, Shanghai 201315, China, Shanghai Research Center for Quantum Sciences, Shanghai 201315, China]    345
[Shanghai Branch, CAS Center for Excellence in Quantum Information and Quantum Physics, University of Science and Technology of China, Shanghai 201315, China, Shanghai Research Center for Qu

In [65]:
dftriple[['paper_id','paper_raw_affiliation_strings']].head()

,paper_id,paper_raw_affiliation_strings
0,https://openalex.org/W3189250281,"[Theoretical Division, Los Alamos National Lab..."
1,https://openalex.org/W3189250281,"[Theoretical Division, Los Alamos National Lab..."
2,https://openalex.org/W3189250281,"[Google Quantum AI Team, Venice, CA, USA]"
3,https://openalex.org/W3189250281,"[Department of Materials, University of Oxford..."
4,https://openalex.org/W3189250281,"[NTT Secure Platform Laboratories, NTT Corpora..."


In [66]:
# Group by 'paper_id' and concatenate 'paper_raw_affiliation_strings'
grouped = dftriple.groupby('paper_id')['paper_raw_affiliation_strings'].apply(lambda x: list(set([item for sublist in x for item in sublist]))).reset_index()

# Convert the series back to a dictionary
pap_affils_dict = grouped.set_index('paper_id')['paper_raw_affiliation_strings'].to_dict()

In [67]:
#paper_affils_dict

In [68]:
#pap_affils_dict = dftriple.groupby('paper_id')['paper_raw_affiliation_strings'].\
#apply(lambda x: ' | '.join(x.tolist()))

#pap_affils_dict = dftriple.set_index('paper_id')['paper_raw_affiliation_strings'].to_dict()
import itertools

#pap_affils_dict = dftriple.set_index('paper_id')['paper_raw_affiliation_strings']



      

In [69]:
type(pap_affils_dict)

dict

In [70]:
#pap_affils_dict.head()

In [71]:
pap_authors_dict = dftriple.groupby('paper_id')['paper_author_display_name'].apply(lambda x: x.values)

In [72]:
pap_authors_dict

paper_id
https://openalex.org/W127096118                  [Martine Ceberio, Vladik Kreinovich]
https://openalex.org/W1521266393    [Alexandre Blais, Alexandre Blais, Arne L. Gri...
https://openalex.org/W1810356272                [Andris Ambainis, Abuzer Yakaryılmaz]
https://openalex.org/W2255489834                                     [Sazzad Hossain]
https://openalex.org/W2266960184                                      [Guang Ping He]
                                                          ...                        
https://openalex.org/W4399045763    [Kostas Vilkelis, Antonio L. R. Manesco, Juan ...
https://openalex.org/W4399075058    [Kumin Kang, Jaber Derakhshandeh, Christian We...
https://openalex.org/W4399107520                          [Mahmud Azam, Steven Rayan]
https://openalex.org/W629978956                     [Thomas Vidick, Stephanie Wehner]
https://openalex.org/W927852154                            [Tobias Graß, Tobias Graß]
Name: paper_author_display_name, Length: 1493

In [73]:
type(pap_authors_dict)

pandas.core.series.Series

In [74]:
#dfinfo.head()

In [75]:
dfinfo['affil_list'] = dfinfo['id'].map(pap_affils_dict)

In [76]:
dfinfo['affil_list'].head().iloc[4]

['Department of Physics, University of California Berkeley, Berkeley, CA, USA',
 'Institute for Theoretical Physics, University of Innsbruck, Innsbruck, Austria',
 'Department of Physics and Research Laboratory of Electronics, Massachusetts Institute of Technology, Cambridge, MA, USA',
 'Department of Physics, Harvard University, Cambridge, MA, USA']

In [77]:
dfinfo[['id','affil_list']].iloc[4]

id                             https://openalex.org/W3180039530
affil_list    [Department of Physics, University of Californ...
Name: https://openalex.org/W3180039530, dtype: object

In [78]:
dfinfo['author_list'] = dfinfo['id'].map(pap_authors_dict)

In [79]:
dfinfo['wrapped_affil_list'] = dfinfo['affil_list'].apply(str).apply(wrap_it)
dfinfo['wrapped_author_list'] = dfinfo['author_list'].apply(str).apply(wrap_it)

In [80]:
dfinfo['wrapped_keywords'] = dfinfo['keywords'].apply(str).apply(wrap_it)

In [81]:
dfinfo['locations'].iloc[69]

[{'is_oa': False,
  'landing_page_url': 'https://doi.org/10.1038/s41586-021-04292-7',
  'pdf_url': None,
  'source': {'id': 'https://openalex.org/S137773608',
   'display_name': 'Nature',
   'issn_l': '0028-0836',
   'issn': ['0028-0836', '1476-4687'],
   'is_oa': False,
   'is_in_doaj': False,
   'host_organization': 'https://openalex.org/P4310319908',
   'host_organization_name': 'Nature Portfolio',
   'host_organization_lineage': ['https://openalex.org/P4310319908',
    'https://openalex.org/P4310319965'],
   'host_organization_lineage_names': ['Nature Portfolio', 'Springer Nature'],
   'type': 'journal'},
  'license': None,
  'license_id': None,
  'version': None,
  'is_accepted': False,
  'is_published': False},
 {'is_oa': False,
  'landing_page_url': 'https://hal.science/hal-04023390',
  'pdf_url': None,
  'source': {'id': 'https://openalex.org/S4306402512',
   'display_name': 'HAL (Le Centre pour la Communication Scientifique Directe)',
   'issn_l': None,
   'issn': None,
   'is

In [82]:
def get_source_name(loc_list):
    """
    grab the first item in the list;
    retturn the display name
    """
    try:
        primary = loc_list[0]
        return primary["source"]["display_name"]
    except:
        return None

def get_source_type(loc_list):
    """
    grab the first item in the list;
    return the source type
    """
    try:
        primary = loc_list[0]
        return primary["source"]["type"]
    except:
        return None

In [83]:
dfinfo["source"] = dfinfo["locations"].apply(get_source_name)
dfinfo["source_type"] = dfinfo["locations"].apply(get_source_type)

In [84]:
dfinfo["source"].value_counts()


arXiv (Cornell University)                                              7969
Physical review. A/Physical review, A                                    814
Physical review letters                                                  435
Research Square (Research Square)                                        360
Physical review research                                                 344
                                                                        ... 
Eastern-European journal of enterprise technologies                        1
Automatisierungstechnik                                                    1
Journal of applied science and technology trends                           1
Izvestiâ Tulʹskogo gosudarstvennogo universiteta. Nauki o Zemle            1
International Journal of Applied Mathematics and Theoretical Physics       1
Name: source, Length: 1817, dtype: int64

In [85]:
dfinfo["source_type"].value_counts()

journal           10416
repository         8554
book series         845
conference          825
ebook platform      311
Name: source_type, dtype: int64

In [86]:
dfinfo[dfinfo["source_type"] == "conference"]

,x,y,id,title,doi,cluster,probability,publication_date,grants,locations,keywords,top_concepts,affil_list,author_list,wrapped_affil_list,wrapped_author_list,wrapped_keywords,source,source_type
id,,,,,,,,,,,,,,,,,,,
https://openalex.org/W4312694312,3.787957,-3.481215,https://openalex.org/W4312694312,Quantum Multi-Agent Reinforcement Learning via...,https://doi.org/10.1109/icdcs54860.2022.00151,105,0.560145,2022-07-01,[{'funder': 'https://openalex.org/F4320320671'...,"[{'is_oa': False, 'landing_page_url': 'https:/...","[Quantum Circuit Design, Variational Quantum C...","[Reinforcement learning, Computer science, Qua...","[Korea University,School of Electrical Enginee...","[Won Joon Yun, Yunseok Kwak, Jae Pyoung Kim, H...","['Korea University,School of Electrical<br>Eng...",['Won Joon Yun' 'Yunseok Kwak' 'Jae<br>Pyoung ...,"['Quantum Circuit Design', 'Variational<br>Qua...",2022 IEEE 42nd International Conference on Dis...,conference
https://openalex.org/W3169143252,1.752588,-3.548632,https://openalex.org/W3169143252,Quantum Federated Learning with Quantum Data,https://doi.org/10.1109/icassp43922.2022.9746622,20,1.000000,2022-05-23,[{'funder': 'https://openalex.org/F4320306076'...,"[{'is_oa': False, 'landing_page_url': 'https:/...","[Quantum Federated Learning, Quantum, Quantum ...","[Computer science, Quantum machine learning, Q...",[Bradley Department of Electrical and Computer...,"[Mahdi Chehimi, Walid Saad]",['Bradley Department of Electrical and<br>Comp...,['Mahdi Chehimi' 'Walid Saad'],"['Quantum Federated Learning',<br>'Quantum', '...",ICASSP 2022 - 2022 IEEE International Conferen...,conference
https://openalex.org/W4220687857,7.925788,1.517570,https://openalex.org/W4220687857,A Cryo-CMOS Low-Power Semi-Autonomous Qubit St...,https://doi.org/10.1109/isscc42614.2022.9731538,99,1.000000,2022-02-20,[],"[{'is_oa': False, 'landing_page_url': 'https:/...","[Qubit State Controller, Cryo-CMOS Low-Power S...","[Qubit, Quantum computer]","[IBM T.J.Watson Research Center, Yorktown Heig...","[D.J. Frank, Sudipto Chakraborty, Kevin Tien, ...","['IBM T.J.Watson Research Center,<br>Yorktown ...",['D.J. Frank' 'Sudipto Chakraborty'<br>'Kevin ...,"['Qubit State Controller', 'Cryo-CMOS<br>Low-P...",2022 IEEE International Solid- State Circuits ...,conference
https://openalex.org/W3205418336,2.399632,-4.659279,https://openalex.org/W3205418336,Classical-To-Quantum Transfer Learning for Spo...,https://doi.org/10.1109/icassp43922.2022.9747636,107,0.509931,2022-05-23,[],"[{'is_oa': False, 'landing_page_url': 'https:/...","[Command Recognition Based, Spoken Command Rec...",[Computer science],"[Electrical and Computer Engineering, Georgia ...","[Jun Qi, Javier Tejedor]","['Electrical and Computer Engineering,<br>Geor...",['Jun Qi' 'Javier Tejedor'],"['Command Recognition Based', 'Spoken<br>Comma...",ICASSP 2022 - 2022 IEEE International Conferen...,conference
https://openalex.org/W4302888899,3.399366,-0.863298,https://openalex.org/W4302888899,Evolution of Quantum Computing: A Systematic S...,https://doi.org/10.1109/compsac54236.2022.00096,127,0.778360,2022-06-01,[{'funder': 'https://openalex.org/F4320306076'...,"[{'is_oa': False, 'landing_page_url': 'https:/...","[Quantum Computing, Quantum, quantum Computing...","[Quantum computer, Computer science]","[Department Computer Science, Kennesaw State U...","[Paramita Basak Upama, Md Jobair Hossain Faruk...","['Department Computer Science, Kennesaw<br>Sta...",['Paramita Basak Upama' 'Md Jobair<br>Hossain ...,"['Quantum Computing', 'Quantum',<br>'quantum C...","2022 IEEE 46th Annual Computers, Software, and...",conference
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://openalex.org/W4315815732,-1.055946,0.914968,https://openalex.org/W4315815732,PVCLS-SI: Isogeny-based Certificateless Signat...,https://doi.org/10.1109/icicn56848.2022.10006501,-1,0.000000,2022-08-23,[{'funder': 'https://openalex.org/F4320320885'...,"[{'is_oa': False, 'landing_page_url': 'https:/..."

In [87]:
dfinfo.columns

Index(['x', 'y', 'id', 'title', 'doi', 'cluster', 'probability',
       'publication_date', 'grants', 'locations', 'keywords', 'top_concepts',
       'affil_list', 'author_list', 'wrapped_affil_list',
       'wrapped_author_list', 'wrapped_keywords', 'source', 'source_type'],
      dtype='object')

In [88]:
dftriple.columns

Index(['id', 'display_name', 'ror', 'country_code', 'type', 'lineage',
       'paper_id', 'paper_raw_affiliation_strings', 'paper_author_position',
       'paper_doi', 'paper_title', 'paper_abstract', 'paper_publication_date',
       'paper_publication_year', 'paper_grants', 'paper_locations',
       'paper_is_corrresponding', 'paper_x', 'paper_y', 'paper_cluster',
       'paper_cluster_score', 'paper_author_id', 'paper_author_display_name',
       'paper_author_orcid'],
      dtype='object')

In [89]:
dfinfo.to_pickle('updatejammingdfinfo2d.pkl')

In [90]:
#dftriple['paper_grants'].value_counts()

In [91]:
def get_funder_names(funder_list):
    """
    funder_list is a list of dictionaries
    with three keys; return the list of 
    unique **funder_display_name**
    values
    """
    try:
        funder_names = list(set([f['funder_display_name'] for f in funder_list]))
        return funder_names    
    except:
        return []
        

In [92]:
dftriple["source"] = dftriple["paper_locations"].apply(get_source_name)
dftriple["source_type"] = dftriple["paper_locations"].apply(get_source_type)
dftriple["funder_list"] = dftriple["paper_grants"].apply(get_funder_names)

In [93]:
dftriple.to_pickle('updatejammingdftriple2d.pkl')

In [94]:
dftriple['source'].value_counts()

Physical review. A/Physical review, A                                                                  4637
Physical review letters                                                                                4293
arXiv (Cornell University)                                                                             2440
Physical review applied                                                                                2281
Research Square (Research Square)                                                                      2188
                                                                                                       ... 
2021 IEEE Conference on Control Technology and Applications (CCTA)                                        1
2022 12th International Conference on Advanced Computer Information Technologies (ACIT)                   1
Iskusstvennye obŝestva                                                                                    1
International Journal For Mu

In [95]:
dftriple['source_type'].value_counts()

journal           59519
repository         5392
conference         4208
book series        2594
ebook platform      419
Name: source_type, dtype: int64

In [96]:
def get_journals_cluster_sort(dc:pd.DataFrame, cl:int):
    """
    restricts the dataframe dc to cluster value cl
    and returns the results grouped by source (where
    source_type == 'journal') sorted
    by the some of probablity descending
    """
    dg = dc[dc['paper_cluster'] == cl].copy()
    print(cl)
    dv = dg[dg['source_type'] == 'journal'].groupby(['source'])['paper_cluster_score'].sum().to_frame()
    dv.sort_values('paper_cluster_score', ascending=False, inplace=True)
    kw = centroids[centroids.cluster == cl]['keywords'].iloc[0]
    return dv, kw

In [97]:
def get_conferences_cluster_sort(dc:pd.DataFrame, cl:int):
    """
    restricts the dataframe dc to cluster value cl
    and returns the results grouped by source (where
    source_type == 'journal') sorted
    by the some of probablity descending
    """
    dg = dc[dc['paper_cluster'] == cl].copy()
    print(cl)
    dv = dg[dg['source_type'] == 'conference'].groupby(['source'])['paper_cluster_score'].sum().to_frame()
    dv.sort_values('paper_cluster_score', ascending=False, inplace=True)
    kw = centroids[centroids.cluster == cl]['keywords'].iloc[0]
    return dv, kw

In [98]:
dv84, kw84 = get_journals_cluster_sort(dftriple, 1)
print(kw84)
dv84.head(10)

1
['Annual Meeting', 'Division of Atomic', 'APS Division', 'Optical Physics', 'Molecular and Optical', 'Presenter Help Desk', 'Virtual Presenter', 'June', 'USA', 'optical', 'Annual', 'Atomic', 'Molecular', 'Orlando', 'Desk', 'Meeting', 'APS', 'Division', 'Physics', 'Virtual']


,paper_cluster_score
source,


# Country - Country Collaborations

want to report back though which countries are involved as well. ok.

In [99]:
def get_country_collaborations_sort(dc:pd.DataFrame, cl:int):
    """
    resticts the dataframe dc to cluster value cl
    and returns the results of paper_id s where there is 
    more than one country_code
    """
    dg = dc[dc['paper_cluster'] == cl].copy()
    dv = dg.groupby('paper_id')['country_code'].apply(lambda x: len(set(x.values))).to_frame()
    dc = dg.groupby('paper_id')['country_code'].apply(lambda x: list(set(x.values))).to_frame()
    dc.columns = ['collab_countries']
    dv.columns = ['country_count']
    dv['collab_countries'] = dc['collab_countries']
    dv.sort_values('country_count',ascending=False, inplace=True)
    di = dfinfo.loc[dv.index].copy()
    di['country_count'] = dv['country_count']
    di['collab_countries'] = dv['collab_countries']
    return di[di['country_count'] > 1]

In [100]:
dv = get_country_collaborations_sort(dftriple, 0)
dv

,x,y,id,title,doi,cluster,probability,publication_date,grants,locations,...,top_concepts,affil_list,author_list,wrapped_affil_list,wrapped_author_list,wrapped_keywords,source,source_type,country_count,collab_countries
paper_id,,,,,,,,,,,,,,,,,,,,,
https://openalex.org/W4285294174,-0.466672,7.741816,https://openalex.org/W4285294174,Quantum-Musical Explorations on $$\mathbb {Z}_n$$,https://doi.org/10.1007/978-3-031-07015-0_32,0,1.0,2022-01-01,[],"[{'is_oa': False, 'landing_page_url': 'https:/...",...,[],[Bernstein Center for Computational Neuroscien...,"[Thomas Noll, Peter beim Graben]",['Bernstein Center for Computational<br>Neuros...,['Thomas Noll' 'Peter beim Graben'],"['Quantum-Musical Explorations',<br>'Quantum H...",Lecture notes in computer science,book series,2,"[ES, DE]"


In [101]:
dfinfo.head()

,x,y,id,title,doi,cluster,probability,publication_date,grants,locations,keywords,top_concepts,affil_list,author_list,wrapped_affil_list,wrapped_author_list,wrapped_keywords,source,source_type
id,,,,,,,,,,,,,,,,,,,
https://openalex.org/W3189250281,6.192506,-3.169462,https://openalex.org/W3189250281,Variational quantum algorithms,https://doi.org/10.1038/s42254-021-00348-9,145,1.000000,2021-08-12,[],"[{'is_oa': False, 'landing_page_url': 'https:/...","[quantum, Variational quantum algorithms, Vari...","[Quantum computer, Computer science, Quantum a...","[Quantum Science Center, Oak Ridge, TN, USA, G...","[M. Cerezo, Andrew Arrasmith, Ryan Babbush, Si...","['Quantum Science Center, Oak Ridge, TN,<br>US...",['M. Cerezo' 'Andrew Arrasmith' 'Ryan<br>Babbu...,"['quantum', 'Variational quantum<br>algorithms...",Nature reviews physics,journal
https://openalex.org/W1521266393,6.635708,2.777389,https://openalex.org/W1521266393,Circuit quantum electrodynamics,https://doi.org/10.1103/revmodphys.93.025005,70,0.665913,2021-05-19,[{'funder': 'https://openalex.org/F4320326644'...,"[{'is_oa': False, 'landing_page_url': 'https:/...","[Circuit quantum electrodynamics, quantum elec...","[Physics, Circuit quantum electrodynamics, Cav...",[Institut quantique and Département de Physiqu...,"[Alexandre Blais, Alexandre Blais, Arne L. Gri...",['Institut quantique and Département de<br>Phy...,['Alexandre Blais' 'Alexandre Blais'<br>'Arne ...,"['Circuit quantum electrodynamics',<br>'quantu...",Reviews of modern physics,journal
https://openalex.org/W4213212652,5.223140,-0.724157,https://openalex.org/W4213212652,Noisy intermediate-scale quantum algorithms,https://doi.org/10.1103/revmodphys.94.015004,143,0.980064,2022-02-15,[{'funder': 'https://openalex.org/F4320306084'...,"[{'is_oa': False, 'landing_page_url': 'https:/...","[Noisy intermediate-scale quantum, intermediat...","[Computer science, Quantum computer, Qubit]","[Centre for Quantum Technologies, National Uni...","[Kishor Bharti, Kishor Bharti, Alba Cervera-Li...","['Centre for Quantum Technologies,<br>National...",['Kishor Bharti' 'Kishor Bharti' 'Alba<br>Cerv...,"['Noisy intermediate-scale quantum',<br>'inter...",Reviews of modern physics,journal
https://openalex.org/W3173899054,5.762541,-0.747414,https://openalex.org/W3173899054,Strong Quantum Computational Advantage Using a...,https://doi.org/10.1103/physrevlett.127.180501,-1,0.000000,2021-10-25,[{'funder': 'https://openalex.org/F4320309612'...,"[{'is_oa': False, 'landing_page_url': 'https:/...","[Quantum Computational Advantage, Superconduct...","[Quantum computer, Qubit, Computer science]",[Shanghai Research Center for Quantum Sciences...,"[Yulin Wu, Yulin Wu, Yulin Wu, Wan-Su Bao, Shu...",['Shanghai Research Center for Quantum<br>Scie...,['Yulin Wu' 'Yulin Wu' 'Yulin Wu' 'Wan-<br>Su ...,"['Quantum Computational Advantage',<br>'Superc...",Physical review letters,journal
https://openalex.org/W3180039530,8.667332,2.101456,https://openalex.org/W3180039530,Quantum phases of matter on a 256-atom program...,https://doi.org/10.1038/s41586-021-03582-4,78,0.847872,2021-07-07,[],"[{'is_oa': False, 'landing_page_url': 'https:/...","[Quantum, programmable quantum simulator, prog...","[Quantum simulator, Quantum]","[Department of Physics, University of Californ...","[Sepehr Ebadi, Tout T. Wang, Harry Levine, Ale...","['Department of Physics, University of<br>Cali...",['Sepehr Ebadi' 'Tout T. Wang' 'Harry<br>Levin...,"['Quantum', 'programmable quantum<br>simulator...",Nature,journal


In [102]:
dftriple.head()

,id,display_name,ror,country_code,type,lineage,paper_id,paper_raw_affiliation_strings,paper_author_position,paper_doi,...,paper_x,paper_y,paper_cluster,paper_cluster_score,paper_author_id,paper_author_display_name,paper_author_orcid,source,source_type,funder_list
0,https://openalex.org/I1343871089,Los Alamos National Laboratory,https://ror.org/01e41cf67,US,facility,"[https://openalex.org/I1330989302, https://ope...",https://openalex.org/W3189250281,"[Theoretical Division, Los Alamos National Lab...",first,https://doi.org/10.1038/s42254-021-00348-9,...,6.192506,-3.169462,145,1.0,https://openalex.org/A5074921335,M. Cerezo,https://orcid.org/0000-0002-2757-3170,Nature reviews physics,journal,[]
1,https://openalex.org/I1343871089,Los Alamos National Laboratory,https://ror.org/01e41cf67,US,facility,"[https://openalex.org/I1330989302, https://ope...",https://openalex.org/W3189250281,"[Theoretical Division, Los Alamos National Lab...",middle,https://doi.org/10.1038/s42254-021-00348-9,...,6.192506,-3.169462,145,1.0,https://openalex.org/A5065899096,Andrew Arrasmith,https://orcid.org/0000-0003-2674-9370,Nature reviews physics,journal,[]
2,https://openalex.org/I1291425158,Google (United States),https://ror.org/00njsd438,US,company,"[https://openalex.org/I1291425158, https://ope...",https://openalex.org/W3189250281,"[Google Quantum AI Team, Venice, CA, USA]",middle,https://doi.org/10.1038/s42254-021-00348-9,...,6.192506,-3.169462,145,1.0,https://openalex.org/A5059699271,Ryan Babbush,https://orcid.org/0000-0001-6979-9533,Nature reviews physics,journal,[]
3,https://openalex.org/I40120149,University of Oxford,https://ror.org/052gg0110,GB,education,[https://openalex.org/I40120149],https://openalex.org/W3189250281,"[Department of Materials, University of Oxford...",middle,https://doi.org/10.1038/s42254-021-00348-9,...,6.192506,-3.169462,145,1.0,https://openalex.org/A5012132706,Simon C. Benjamin,https://orcid.org/0000-0002-7766-5348,Nature reviews physics,journal,[]
4,https://openalex.org/I2251713219,NTT (Japan),https://ror.org/00berct97,JP,company,[https://openalex.org/I2251713219],https://openalex.org/W3189250281,"[NTT Secure Platform Laboratories, NTT Corpora...",middle,https://doi.org/10.1038/s42254-021-00348-9,...,6.192506,-3.169462,145,1.0,https://openalex.org/A5082360196,Suguru Endo,https://orcid.org/0000-0002-2317-3751,Nature reviews physics,journal,[]


In [103]:
jamming_concepts

[{'id': 'https://openalex.org/C58053490',
  'wikidata': 'https://www.wikidata.org/wiki/Q176555',
  'display_name': 'Quantum computer',
  'relevance_score': 40565.902,
  'level': 3,
  'description': 'theoretical computation device relying on quantum mechanics',
  'works_count': 62241,
  'cited_by_count': 1296288,
  'summary_stats': {'2yr_mean_citedness': 3.0585373387906403,
   'h_index': 432,
   'i10_index': 14982},
  'ids': {'openalex': 'https://openalex.org/C58053490',
   'wikidata': 'https://www.wikidata.org/wiki/Q176555',
   'mag': '58053490',
   'wikipedia': 'https://en.wikipedia.org/wiki/Quantum%20computing'},
  'image_url': 'https://upload.wikimedia.org/wikipedia/commons/6/60/IBM_Q_system_%28Fraunhofer_2%29.jpg',
  'image_thumbnail_url': 'https://upload.wikimedia.org/wikipedia/commons/thumb/6/60/IBM_Q_system_%28Fraunhofer_2%29.jpg/100px-IBM_Q_system_%28Fraunhofer_2%29.jpg',
  'international': {'display_name': {'ar': 'حاسوب كمومي',
    'ast': 'Computación cuántica',
    'az': 'Kva

# Co-authorship Network

Streamlit with pyvis: https://towardsdatascience.com/how-to-deploy-interactive-pyvis-network-graphs-on-streamlit-6c401d4c99db

the data source is dftriple; let a user interactively select which type of graph , the selection of node types, to display. otherwise its just too too much.

Can display works and authors; construct that first:

In [104]:
dftriple.columns

Index(['id', 'display_name', 'ror', 'country_code', 'type', 'lineage',
       'paper_id', 'paper_raw_affiliation_strings', 'paper_author_position',
       'paper_doi', 'paper_title', 'paper_abstract', 'paper_publication_date',
       'paper_publication_year', 'paper_grants', 'paper_locations',
       'paper_is_corrresponding', 'paper_x', 'paper_y', 'paper_cluster',
       'paper_cluster_score', 'paper_author_id', 'paper_author_display_name',
       'paper_author_orcid', 'source', 'source_type', 'funder_list'],
      dtype='object')

group dftriple by paper_id and get a list of all the paper_author_id values. and then from that list get all distinct subsets of two paper_author_ids. accumulate that list. and then we will haave a weighted undirected graph.

file://wsl.localhost/Ubuntu/home/davidd/2023/SWITCHBOARD/switchboard-mitigations-sort/graphvizmaker.html

In [105]:
import networkx as nx
from pyvis.network import Network
import igraph as ig # for getting a layout w/o relying on slow pyvis physics 

In [106]:
dftriple.columns

Index(['id', 'display_name', 'ror', 'country_code', 'type', 'lineage',
       'paper_id', 'paper_raw_affiliation_strings', 'paper_author_position',
       'paper_doi', 'paper_title', 'paper_abstract', 'paper_publication_date',
       'paper_publication_year', 'paper_grants', 'paper_locations',
       'paper_is_corrresponding', 'paper_x', 'paper_y', 'paper_cluster',
       'paper_cluster_score', 'paper_author_id', 'paper_author_display_name',
       'paper_author_orcid', 'source', 'source_type', 'funder_list'],
      dtype='object')

In [107]:
dfinfo.columns

Index(['x', 'y', 'id', 'title', 'doi', 'cluster', 'probability',
       'publication_date', 'grants', 'locations', 'keywords', 'top_concepts',
       'affil_list', 'author_list', 'wrapped_affil_list',
       'wrapped_author_list', 'wrapped_keywords', 'source', 'source_type'],
      dtype='object')

In [108]:
dfinfo["funder_list"] = dfinfo["grants"].apply(get_funder_names)
dfinfo["wrapped_funder_list"] = dfinfo["funder_list"].apply(str).apply(wrap_it)

In [109]:
dfinfo.to_pickle('updatejammingdfinfo2d.pkl')

In [110]:
dfinfo[['id','keywords','wrapped_keywords','wrapped_funder_list']].head()

,id,keywords,wrapped_keywords,wrapped_funder_list
id,,,,
https://openalex.org/W3189250281,https://openalex.org/W3189250281,"[quantum, Variational quantum algorithms, Vari...","['quantum', 'Variational quantum<br>algorithms...",[]
https://openalex.org/W1521266393,https://openalex.org/W1521266393,"[Circuit quantum electrodynamics, quantum elec...","['Circuit quantum electrodynamics',<br>'quantu...","['Army Research Office', 'Natural<br>Sciences ..."
https://openalex.org/W4213212652,https://openalex.org/W4213212652,"[Noisy intermediate-scale quantum, intermediat...","['Noisy intermediate-scale quantum',<br>'inter...","['Google', 'Natural Sciences and<br>Engineerin..."
https://openalex.org/W3173899054,https://openalex.org/W3173899054,"[Quantum Computational Advantage, Superconduct...","['Quantum Computational Advantage',<br>'Superc...","['Chinese Academy of Sciences', 'USTC<br>Cente..."
https://openalex.org/W3180039530,https://openalex.org/W3180039530,"[Quantum, programmable quantum simulator, prog...","['Quantum', 'programmable quantum<br>simulator...",[]


In [111]:
kw_dict = dfinfo['keywords'].to_dict()

In [112]:
dftriple[['source','source_type']].head()

,source,source_type
0,Nature reviews physics,journal
1,Nature reviews physics,journal
2,Nature reviews physics,journal
3,Nature reviews physics,journal
4,Nature reviews physics,journal


In [113]:
dftriple.head()

,id,display_name,ror,country_code,type,lineage,paper_id,paper_raw_affiliation_strings,paper_author_position,paper_doi,...,paper_x,paper_y,paper_cluster,paper_cluster_score,paper_author_id,paper_author_display_name,paper_author_orcid,source,source_type,funder_list
0,https://openalex.org/I1343871089,Los Alamos National Laboratory,https://ror.org/01e41cf67,US,facility,"[https://openalex.org/I1330989302, https://ope...",https://openalex.org/W3189250281,"[Theoretical Division, Los Alamos National Lab...",first,https://doi.org/10.1038/s42254-021-00348-9,...,6.192506,-3.169462,145,1.0,https://openalex.org/A5074921335,M. Cerezo,https://orcid.org/0000-0002-2757-3170,Nature reviews physics,journal,[]
1,https://openalex.org/I1343871089,Los Alamos National Laboratory,https://ror.org/01e41cf67,US,facility,"[https://openalex.org/I1330989302, https://ope...",https://openalex.org/W3189250281,"[Theoretical Division, Los Alamos National Lab...",middle,https://doi.org/10.1038/s42254-021-00348-9,...,6.192506,-3.169462,145,1.0,https://openalex.org/A5065899096,Andrew Arrasmith,https://orcid.org/0000-0003-2674-9370,Nature reviews physics,journal,[]
2,https://openalex.org/I1291425158,Google (United States),https://ror.org/00njsd438,US,company,"[https://openalex.org/I1291425158, https://ope...",https://openalex.org/W3189250281,"[Google Quantum AI Team, Venice, CA, USA]",middle,https://doi.org/10.1038/s42254-021-00348-9,...,6.192506,-3.169462,145,1.0,https://openalex.org/A5059699271,Ryan Babbush,https://orcid.org/0000-0001-6979-9533,Nature reviews physics,journal,[]
3,https://openalex.org/I40120149,University of Oxford,https://ror.org/052gg0110,GB,education,[https://openalex.org/I40120149],https://openalex.org/W3189250281,"[Department of Materials, University of Oxford...",middle,https://doi.org/10.1038/s42254-021-00348-9,...,6.192506,-3.169462,145,1.0,https://openalex.org/A5012132706,Simon C. Benjamin,https://orcid.org/0000-0002-7766-5348,Nature reviews physics,journal,[]
4,https://openalex.org/I2251713219,NTT (Japan),https://ror.org/00berct97,JP,company,[https://openalex.org/I2251713219],https://openalex.org/W3189250281,"[NTT Secure Platform Laboratories, NTT Corpora...",middle,https://doi.org/10.1038/s42254-021-00348-9,...,6.192506,-3.169462,145,1.0,https://openalex.org/A5082360196,Suguru Endo,https://orcid.org/0000-0002-2317-3751,Nature reviews physics,journal,[]


In [114]:
dc = dftriple[dftriple['paper_cluster'] == 10].copy()
dc.shape

(11, 27)

In [115]:
[x for row in dc['funder_list'].tolist() for x in row]

[]

In [116]:
dftriple['funder_list'].value_counts().head()

[]                                                                                                        44028
[National Natural Science Foundation of China]                                                             2628
[National Science Foundation]                                                                              1185
[National Natural Science Foundation of China, National Key Research and Development Program of China]      675
[U.S. Department of Energy]                                                                                 602
Name: funder_list, dtype: int64

In [117]:
kw_dict = dfinfo['keywords'].to_dict()

# add in the affiliations as nodes as well; that row, author, paper, affil. all three get links. ok.
def create_nx_graph(df: pd.DataFrame, cl:int) -> nx.Graph:
    """
    takes the dataframe df, and creates the undirected graph
    from the source and target columns for each row.
    """
    g = nx.Graph() # dc['paper_cluster'] == cl
    dc = df[df['paper_cluster'] == cl]
    author_counts = dc['paper_author_id'].tolist()
    author_counts_dict = {c:author_counts.count(c) for c in author_counts}
    affiliation_counts = dc['id'].tolist()
    affiliation_counts_dict = {c:affiliation_counts.count(c) for c in affiliation_counts}
    source_counts = dc['source'].tolist()
    source_counts_dict = {c:source_counts.count(c) for c in source_counts}
    funder_counts = [x for row in dc['funder_list'].tolist() for x in row]
    funder_counts_dict = {c:funder_counts.count(c) for c in funder_counts}
    for index, row in df[df['paper_cluster'] == cl].iterrows():
        g.add_node(row['paper_id'], group='work', title=row['paper_title'])
        g.add_node(row['paper_author_id'], title=row['paper_author_display_name'],
                   group='author',value = author_counts_dict[row['paper_author_id']])
        g.add_node(row['id'], group='affiliation',
                   title=row['display_name'] + '\n' + row['country_code'],
                  value = affiliation_counts_dict[row['id']])
        if row['source']:
            g.add_node(row['source'], group=row['source_type'],
                      title=row['source'] + ' :\n ' + row['source_type'],
                      value=source_counts_dict[row['source']])
            g.add_edge(
                row['paper_id'],
                row['source'],
                title=row['paper_title'] + ' :\n ' + str(row['paper_publication_date']) +  \
                ' :\n' + row['source'] + ' :\n ' + \
                row['source_type'],
              #  weight = df[(df['paper_id'] == row['paper_id']) & \
              #              (df['source'] == row['source'])]['paper_cluster_score'].sum()
               # weight = row['paper_cluster_score']
            )
            g.add_edge(
                row['paper_author_id'],
                row['source'],
                title=row['paper_author_display_name'] + ':\n' + row['source'],
             #   weight = df[(df['paper_author_id'] == row['paper_author_id']) & \
              #              (df['source'] == row['source'])]['paper_cluster_score'].sum()
               # weight = row['paper_cluster_score']
            )
        if len(row['funder_list']) > 0:
            for f in row['funder_list']:
                g.add_node(f, group='funder',
                          title=str(f),
                          value = founder_counts_dict[f]),
                g.add_edge(
                       row['paper_id'],
                       f,
                       title=row['paper_title'] + ':\n ' +  str(row['paper_publication_date']) + \
                       ' :\n' + str(f),
                  #  weight = row['paper_cluster_score']
                   )
                g.add_edge(
                       f,
                       row['paper_author_id'],
                       title=row['paper_author_display_name'] + ' :\n ' + \
                       str(f),
                  #  weight = row['paper_cluster_score']
                       
                   )
                g.add_edge(
                       f,
                       row['id'],
                       title=row['display_name'] + '\n' + row['country_code'] + ' :\n ' + \
                       str(f)  ,
                  #  weight = row['paper_cluster_score']
                   )  
                if row["source"]:
                    g.add_edge(
                        f,
                        row["source"],
                        title=row["source"] + ' :\n' + str(f),
                     #   weight = row['paper_cluster_score']
                    )
        g.nodes[row['paper_id']]['title'] = (
            row['paper_title'] + ' :\n ' + str(row['paper_publication_date'] + ':\n' + 
            '\n'.join(kw_dict[row['paper_id']]))
        )
        g.nodes[row['paper_author_id']]['title'] = (
            row['paper_author_display_name']
        )
        g.add_edge(
            row['paper_id'],
            row['paper_author_id'],
        title=row['paper_title'] + ' :\n ' + row['paper_author_display_name'] + ' :\n ' + \
            row['paper_raw_affiliation_string'],
         #   weight = row['paper_cluster_score']
        )
        g.add_edge(
            row['paper_author_id'],
            row['id'],
            title=row['paper_author_display_name'] + ' :\n ' + \
            row['display_name'] + ' :\n ' + row['country_code'],
          #  weight = row['paper_cluster_score']
        )
        g.add_edge(
            row['paper_id'],
            row['id'],
            title=row['paper_title'] + ' :\n ' + str(row['paper_publication_date']) + ':\n' + 
            row['display_name'] + ' :\n ' + row['country_code'],
         #   weight = row['paper_cluster_score']
        )
        
    g_ig = ig.Graph.from_networkx(g) # assign 'x', and 'y' to g before returning
    #layout = g_ig.layout_auto()
    #layout = g_ig.layout_davidson_harel()
    layout = g_ig.layout_umap(min_dist = 2, epochs = 500)
    # https://igraph.org/python/tutorial/0.9.6/visualisation.html
    coords = layout.coords
    allnodes = list(g.nodes())
    coords_dict = {allnodes[i]:(coords[i][0], coords[i][1]) for i in range(len(allnodes))}
    for i in g.nodes():
        g.nodes[i]['x'] = 250 * coords_dict[i][0] # the scale factor needed 
        g.nodes[i]['y'] = 250 * coords_dict[i][1]
    return g
                

In [118]:
def create_pyvis_html(cl: int, filename: str = "pyvis_coauthorships_graph.html"):
    """
    wrapper function that calls create_nx_graph to finally 
    produce an interactive pyvis standalone html file
    """
    g_nx = create_nx_graph(dftriple, cl);
    h = Network(height="1000px",
          #  heading="Mitigations and Techniques Relationships",
                width="100%",
                cdn_resources="remote", # can grab the visjs library to make this local if needed
            # probably should
                bgcolor="#222222",
            neighborhood_highlight=True,
              # default_node_size=1,
                font_color="white",
                directed=False,
               # select_menu=True,
                filter_menu=True,
                notebook=False,
               )
    #h.repulsion()
    h.from_nx(g_nx, show_edge_weights=False)
    #h.barnes_hut()
    #h.repulsion(node_distance=40,
    #            central_gravity=-0.2, spring_length=5, spring_strength=0.005, damping=0.09)
    neighbor_map = h.get_adj_list()
   # for node in h.nodes:
   #     if node['group'] == 'author':
   #         a = list(neighbor_map[node["id"]]) # want to insert a "\n" into every third element of a
   #     if node['group'] == 'work':
   #         a = list(neighbor_map[node["id"]])
   #     i = 3
   #     while i < len(a):
   #         a.insert(i, "\n")
   #         i += 4
   #     node["title"] += "\n Neighbors: \n" + " | ".join(a)
   #     node["value"] = len(neighbor_map[node["id"]]) 
# "physics": {
#    "enabled": false
#  },
    h.set_options(
    """
const options = {
  "interaction": {
    "navigationButtons": false
  },
 "physics": {
     "enabled": false
 },
  "edges": {
    "color": {
        "inherit": true
    },
    "setReferenceSize": null,
    "setReference": {
        "angle": 0.7853981633974483
    },
    "smooth": {
        "forceDirection": "none"
    }
  }
  }
    """
    )
    #h.show_buttons(filter_=['physics'])
  #  h.barnes_hut()
    #h.repulsion()
    try:
        path = './tmp'
        h.save_graph(f"{path}/{filename}")
        HtmlFile = open(f"{path}/{filename}","r",
                        encoding='utf-8')
    except:
        h.save_graph(f"{filename}")
        HtmlFile = open(f"{filename}", "r",
                        encoding="utf-8")
    return h

In [119]:
dfinfo.shape

(24359, 21)

In [120]:
dfinfo.columns

Index(['x', 'y', 'id', 'title', 'doi', 'cluster', 'probability',
       'publication_date', 'grants', 'locations', 'keywords', 'top_concepts',
       'affil_list', 'author_list', 'wrapped_affil_list',
       'wrapped_author_list', 'wrapped_keywords', 'source', 'source_type',
       'funder_list', 'wrapped_funder_list'],
      dtype='object')

In [122]:
dfinfo[['author_list']].head()

,author_list
id,
https://openalex.org/W3189250281,"[M. Cerezo, Andrew Arrasmith, Ryan Babbush, Si..."
https://openalex.org/W1521266393,"[Alexandre Blais, Alexandre Blais, Arne L. Gri..."
https://openalex.org/W4213212652,"[Kishor Bharti, Kishor Bharti, Alba Cervera-Li..."
https://openalex.org/W3173899054,"[Yulin Wu, Yulin Wu, Yulin Wu, Wan-Su Bao, Shu..."
https://openalex.org/W3180039530,"[Sepehr Ebadi, Tout T. Wang, Harry Levine, Ale..."


In [123]:
#del dfinfo['authors_list']

In [124]:
dfinfo.shape

(24359, 21)

In [125]:
dfinfo[['cluster','probability','publication_date']].head()

,cluster,probability,publication_date
id,,,
https://openalex.org/W3189250281,145,1.000000,2021-08-12
https://openalex.org/W1521266393,70,0.665913,2021-05-19
https://openalex.org/W4213212652,143,0.980064,2022-02-15
https://openalex.org/W3173899054,-1,0.000000,2021-10-25
https://openalex.org/W3180039530,78,0.847872,2021-07-07


In [126]:
dftime = dfinfo[['cluster','probability','publication_date']].copy()

In [127]:
dftime['publication_datetime'] = pd.to_datetime(dftime['publication_date'])

In [128]:
dftime.head()

,cluster,probability,publication_date,publication_datetime
id,,,,
https://openalex.org/W3189250281,145,1.000000,2021-08-12,2021-08-12
https://openalex.org/W1521266393,70,0.665913,2021-05-19,2021-05-19
https://openalex.org/W4213212652,143,0.980064,2022-02-15,2022-02-15
https://openalex.org/W3173899054,-1,0.000000,2021-10-25,2021-10-25
https://openalex.org/W3180039530,78,0.847872,2021-07-07,2021-07-07


In [129]:
def get_time_series(dg, cl:int):
    """
    takes dg and the cluster number cl
    and returns a time series chart
    by month, y-axis is the article count
    """
    dftime = dg[dg.cluster == cl][['cluster','probability','publication_date']].copy()
    dftime['date'] = pd.to_datetime(dftime['publication_date'])
    dftime.sort_values('date', inplace=True)
    #by_month = pd.to_datetime(dftime['date']).dt.to_period('M').value_counts().sort_index()
    #by_month.index = pd.PeriodIndex(by_month.index)
    #df_month = by_month.rename_axis('month').reset_index(name='counts')
    return dftime

In [130]:
dfinfo.cluster.value_counts().head()

-1      7312
 38     1750
 108     862
 106     803
 29      599
Name: cluster, dtype: int64

In [131]:
df_month_21 = get_time_series(dfinfo, 12)
df_month_21.head()

,cluster,probability,publication_date,date
id,,,,
https://openalex.org/W3130961928,12,1.0,2021-02-01,2021-02-01
https://openalex.org/W3137382153,12,1.0,2021-03-01,2021-03-01
https://openalex.org/W3153068134,12,1.0,2021-05-07,2021-05-07
https://openalex.org/W3182215211,12,1.0,2021-11-12,2021-11-12
https://openalex.org/W4225440242,12,1.0,2021-12-30,2021-12-30


In [132]:
import altair as alt
#alt.data_transformers.enable("data_server")

In [133]:
alt.Chart(df_month_21).mark_line().transform_fold(
    ['probability']
).encode(
    x = 'yearmonth(date):T',
    y = 'sum(value):Q',
    color='key:N'
)

alt.Chart(...)

In [134]:
sources_list = dftriple['source'].unique().tolist()
type(sources_list), len(sources_list)

(list, 1556)

In [135]:
Sources().random()

{'id': 'https://openalex.org/S4210209619',
 'issn_l': '0791-9417',
 'issn': ['0791-9417'],
 'display_name': 'Classics Ireland',
 'host_organization': None,
 'host_organization_name': None,
 'host_organization_lineage': [],
 'works_count': 187,
 'cited_by_count': 2766,
 'summary_stats': {'2yr_mean_citedness': 0.0, 'h_index': 32, 'i10_index': 57},
 'is_oa': False,
 'is_in_doaj': False,
 'ids': {'openalex': 'https://openalex.org/S4210209619',
  'issn_l': '0791-9417',
  'issn': ['0791-9417'],
  'wikidata': 'https://www.wikidata.org/entity/Q15754232',
  'fatcat': 'https://fatcat.wiki/container/6iiyj6bdr5finfhzfb6vbsmxxi'},
 'homepage_url': None,
 'apc_prices': None,
 'apc_usd': None,
 'country_code': 'IE',
 'societies': [],
 'alternate_titles': [],
 'abbreviated_title': None,
 'type': 'journal',
 'topics': [{'id': 'https://openalex.org/T10165',
   'display_name': 'Society and Economy in Ancient Mediterranean Civilizations',
   'count': 95,
   'subfield': {'id': 'https://openalex.org/subfiel

In [136]:
def get_source_json(s:str):
    """
    s is an openalex Sources display_name
    return that Sources object
    """
    source_json = Sources().search_filter(display_name = s).get()
    a = source_json[0]['type']
    if "homepage_url" in source_json[0] and source_json[0]['homepage_url']:
        print(f"{s} has homepage_url and type {source_json[0]['type']}")
        return source_json[0]["homepage_url"]
    else:
        return None

In [137]:
sources_list[5]

'Nature physics'

In [138]:
sj0 = get_source_json(sources_list[5])
sj0

Nature physics has homepage_url and type journal


'http://www.nature.com/nphys/'

In [139]:
def get_display_page_dict(sl:list):
    """
    sl is a list of Sources display_name values
    returns the dictionary mapping
    display_names with homepage_url values."""
    mapping_dict = dict()
    for s in tqdm(sl):
        try:
            mapping_dict[s] = get_source_json(s)
        except:
            pass
    return mapping_dict

In [140]:
source_page_dict = get_display_page_dict(sources_list)

  0%|                                                                        | 1/1556 [00:00<11:34,  2.24it/s]

Nature reviews physics has homepage_url and type journal


  0%|                                                                        | 2/1556 [00:00<12:10,  2.13it/s]

Reviews of modern physics has homepage_url and type journal


  0%|▏                                                                       | 3/1556 [00:01<12:11,  2.12it/s]

...

100%|████████████████████████████████████████████████████████████████████▉| 1554/1556 [12:51<00:00,  2.23it/s]

Informacionno-upravlâûŝie sistemy has homepage_url and type journal


100%|████████████████████████████████████████████████████████████████████▉| 1555/1556 [12:52<00:00,  2.26it/s]

Pramāna has homepage_url and type journal


100%|█████████████████████████████████████████████████████████████████████| 1556/1556 [12:52<00:00,  2.01it/s]


In [142]:
len(source_page_dict)

1349

In [143]:
import pickle

with open("updatesource_page_dict.pkl", "wb") as f:
    pickle.dump(source_page_dict, f)

In [144]:
with open("updatesource_page_dict.pkl", "rb") as f:
    source_dict = pickle.load(f)

In [145]:
def get_journals_cluster_sort(dc:pd.DataFrame, cl:int):
    """
    restricts the dataframe dc to cluster value cl
    and returns the results grouped by source (where
    source_type == 'journal') sorted
    by the some of probablity descending
    """
    dg = dc[dc['paper_cluster'] == cl].copy()
    print(cl)
    dv = dg[dg['source_type'] == 'journal'].groupby(['source'])['paper_cluster_score'].sum().to_frame()
    dv.sort_values('paper_cluster_score', ascending=False, inplace=True)
    dv['journal'] = dv.index
    dv['hompage_url'] = dv['journal'].map(source_dict)
    kw = centroids[centroids.cluster == cl]['keywords'].iloc[0]
    return dv, kw

In [146]:
dv, kw = get_journals_cluster_sort(dftriple, 4)
dv.head()

4


,paper_cluster_score,journal,hompage_url
source,,,
Quantum science and technology,35.0,Quantum science and technology,http://iopscience.iop.org/journal/2058-9565
Applied physics letters,28.842072,Applied physics letters,http://apl.aip.org/
Physical review letters,19.0,Physical review letters,http://journals.aps.org/prl/
"Physical review. A/Physical review, A",15.161374,"Physical review. A/Physical review, A",NaN
Physical review research,7.769893,Physical review research,https://journals.aps.org/prresearch/


In [147]:
def get_conferences_cluster_sort(dc:pd.DataFrame, cl:int):
    """
    restricts the dataframe dc to cluster value cl
    and returns the results grouped by source (where
    source_type == 'journal') sorted
    by the some of probablity descending
    """
    dg = dc[dc['paper_cluster'] == cl].copy()
    print(cl)
    dv = dg[dg['source_type'] == 'conference'].groupby(['source'])['paper_cluster_score'].sum().to_frame()
    dv.sort_values('paper_cluster_score', ascending=False, inplace=True)
    dv['conference'] = dv.index
    dv['homepage_url'] = dv['conference'].map(source_dict)
    kw = centroids[centroids.cluster == cl]['keywords'].iloc[0]
    return dv, kw

In [148]:
dv, kw = get_conferences_cluster_sort(dftriple, 4)
dv.head()

4


,paper_cluster_score,conference,homepage_url
source,,,


In [149]:
"IOP conference series" in source_dict

False

In [150]:
len(source_dict)

1349

In [151]:
affils_list = dftriple['display_name'].unique().tolist()
type(affils_list), len(affils_list)

(list, 4239)

In [152]:
affils_list[0]

'Los Alamos National Laboratory'

In [153]:
 affil_json = Institutions().search_filter(display_name = affils_list[0]).get()

In [154]:
#affil_json

In [155]:
affils_list = dftriple['display_name'].unique().tolist()
type(affils_list), len(affils_list)

affil_json = Institutions().search_filter(display_name = affils_list[1]).get()

def get_affil_json(s:str):
    """
    s is an openalex Sources display_name
    return that Sources object
    """
    affil_json = Institutions().search_filter(display_name = s).get()
    #a = source_json[0]['type']
    if "geo" in affil_json[0]:
       # print(f"{s} has homepage_url and type {source_json[0]['type']}")
        return affil_json[0]["geo"]["latitude"], affil_json[0]["geo"]["longitude"]
    else:
        return None, None
    
def get_display_geo_dict(sl:list):
    """
    sl is a list of Institution display_name values
    returns the dictionary mapping
    display_names with (latitude, longitude) values."""
    mapping_dict = dict()
    for s in tqdm(sl):
        try:
            mapping_dict[s] = get_affil_json(s)
        except:
            pass
    return mapping_dict

affil_geo_dict = get_display_geo_dict(affils_list)


import pickle

with open("updateaffil_geo_dict.pkl", "wb") as f:
    pickle.dump(affil_geo_dict, f)

100%|█████████████████████████████████████████████████████████████████████| 4239/4239 [33:13<00:00,  2.13it/s]


In [156]:
len(affil_geo_dict)

4093

In [157]:
dftriple.columns

Index(['id', 'display_name', 'ror', 'country_code', 'type', 'lineage',
       'paper_id', 'paper_raw_affiliation_strings', 'paper_author_position',
       'paper_doi', 'paper_title', 'paper_abstract', 'paper_publication_date',
       'paper_publication_year', 'paper_grants', 'paper_locations',
       'paper_is_corrresponding', 'paper_x', 'paper_y', 'paper_cluster',
       'paper_cluster_score', 'paper_author_id', 'paper_author_display_name',
       'paper_author_orcid', 'source', 'source_type', 'funder_list'],
      dtype='object')

In [158]:
dftriple[['paper_title','paper_abstract']].head()

,paper_title,paper_abstract
0,Variational quantum algorithms,Applications such as simulating complicated qu...
1,Variational quantum algorithms,Applications such as simulating complicated qu...
2,Variational quantum algorithms,Applications such as simulating complicated qu...
3,Variational quantum algorithms,Applications such as simulating complicated qu...
4,Variational quantum algorithms,Applications such as simulating complicated qu...


In [159]:
dfinfo.columns

Index(['x', 'y', 'id', 'title', 'doi', 'cluster', 'probability',
       'publication_date', 'grants', 'locations', 'keywords', 'top_concepts',
       'affil_list', 'author_list', 'wrapped_affil_list',
       'wrapped_author_list', 'wrapped_keywords', 'source', 'source_type',
       'funder_list', 'wrapped_funder_list'],
      dtype='object')

In [160]:
#affil_geo_dict

need to add the abstract to dfinfo

In [161]:
dftriple[['paper_title','paper_abstract']].head()

,paper_title,paper_abstract
0,Variational quantum algorithms,Applications such as simulating complicated qu...
1,Variational quantum algorithms,Applications such as simulating complicated qu...
2,Variational quantum algorithms,Applications such as simulating complicated qu...
3,Variational quantum algorithms,Applications such as simulating complicated qu...
4,Variational quantum algorithms,Applications such as simulating complicated qu...


In [162]:
dfinfo['abstract'] = dfinfo['title'].map(dftriple.set_index('paper_title')['paper_abstract'].to_dict())

In [163]:
dfinfo[['title','abstract']].head(10)

,title,abstract
id,,
https://openalex.org/W3189250281,Variational quantum algorithms,Applications such as simulating complicated qu...
https://openalex.org/W1521266393,Circuit quantum electrodynamics,This review surveys the development over the l...
https://openalex.org/W4213212652,Noisy intermediate-scale quantum algorithms,A universal fault-tolerant quantum computer th...
https://openalex.org/W3173899054,Strong Quantum Computational Advantage Using a...,Scaling up to a large number of qubits with hi...
https://openalex.org/W3180039530,Quantum phases of matter on a 256-atom program...,Motivated by far-reaching applications ranging...
https://openalex.org/W3216297445,Probing topological spin liquids on a programm...,"Quantum phases with topological order, such as..."
https://openalex.org/W4281891658,Quantum computational advantage with a program...,A quantum computer attains computational advan...
https://openalex.org/W3127319997,Moiré heterostructures as a condensed-matter q...,Twisted van der Waals heterostructures have la...
https://openalex.org/W3153423942,Realization of a multinode quantum network of ...,A three-node quantum network Future quantum ne...


In [164]:
dfinfo.to_pickle('updatejammingdfinfo2d.pkl')

Now try to recreate the pydeck column chart a la DPS-TTU instead of the flat 2d scatter plot. ok.
can also maybe add arcs in between collaborations. maybe. 

In [165]:
dftriple.columns

Index(['id', 'display_name', 'ror', 'country_code', 'type', 'lineage',
       'paper_id', 'paper_raw_affiliation_strings', 'paper_author_position',
       'paper_doi', 'paper_title', 'paper_abstract', 'paper_publication_date',
       'paper_publication_year', 'paper_grants', 'paper_locations',
       'paper_is_corrresponding', 'paper_x', 'paper_y', 'paper_cluster',
       'paper_cluster_score', 'paper_author_id', 'paper_author_display_name',
       'paper_author_orcid', 'source', 'source_type', 'funder_list'],
      dtype='object')

In [166]:
affil_geo_dict

{'Los Alamos National Laboratory': (35.88808, -106.30697),
 'Google (United States)': (37.422955, -122.08406),
 'University of Oxford': (51.75222, -1.25596),
 'NTT (Japan)': (35.689526, 139.75345),
 'Osaka University': (34.69374, 135.50218),
 'Peking University': (39.9075, 116.39723),
 'Quantum Science Center': (36.32002, -87.76545),
 'Canadian Institute for Advanced Research': (43.65538, -79.38645),
 'Université de Sherbrooke': (45.40008, -71.89908),
 'Centre for Quantum Technologies': (1.297461, 103.7803),
 'National University of Singapore': (1.28967, 103.85007),
 'Hefei National Center for Physical Sciences at Nanoscale': (31.8227,
  117.28035),
 'University of Science and Technology of China': (31.86389, 117.28083),
 'Beijing Academy of Quantum Information Sciences': (39.9075, 116.39723),
 'State Key Laboratory of Cryptology': (39.87733, 116.2787),
 'QuantumCTek (China)': (31.821615, 117.21416),
 'Shanghai Institute of Technical Physics': (31.22222, 121.45806),
 'Chinese Academy o

In [167]:
dvaffils, kwwaffils = get_affils_cluster_sort(dftriple, 10)

10


In [168]:
#affil_geo_dict['University of Electronic Science and Technology of China']

In [169]:
dvaffils.head()

,id,display_name,country_code,type,paper_cluster_score,latitude,longitude
0,https://openalex.org/I122411786,California Institute of Technology,US,education,3.0,34.147780,-118.14452
1,https://openalex.org/I98358874,Delft University of Technology,NL,education,3.0,52.006670,4.35556
2,https://openalex.org/I192578771,Rose–Hulman Institute of Technology,US,education,1.0,39.482716,-87.32400
3,https://openalex.org/I78650965,TU Dresden,DE,education,0.611036,51.050890,13.73832
4,https://openalex.org/I124227911,Ben-Gurion University of the Negev,IL,education,0.421383,31.262192,34.80151


ok, can color by type, height and size given by paper_cluster_score

may need to add in the mapbox token. ok.

follow along the treatment here: https://github.com/rs21io/ttu-paths/blob/main/suescoreshypothesis.ipynb

mapbox_token="pk.eyJ1IjoibWFwYm94cnMyMSIsImEiOiJjamdkdTU1MTIwMTM2Mnhxa3Y3ZXZ3eGt3In0.PtflK7MObAbmwY1E__H7Fg"

In [170]:
import pydeck as pdk

view = pdk.data_utils.compute_view(dvaffils[["longitude", "latitude"]])

mapbox_token="pk.eyJ1IjoibWFwYm94cnMyMSIsImEiOiJjamdkdTU1MTIwMTM2Mnhxa3Y3ZXZ3eGt3In0.PtflK7MObAbmwY1E__H7Fg"